In [ ]:
!pip install chainer

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
import time
import copy
import numpy as np
import pandas as pd
import chainer
import chainer.functions as F
import chainer.links as L
from plotly import tools
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot, iplot_mpl
import csv
from google.colab import drive

veri= pd.read_csv("/content/drive/MyDrive/DDos.pcap_ISCX.csv", delimiter=',', skiprows=0, low_memory=False)
veri.head()


In [ ]:
!pip install cupy-cuda101>=7.7.0

In [ ]:
veri[' Label'].replace(['BENIGN','DDoS'],[0,1],inplace=True)

In [ ]:
moddf= veri.dropna()
moddf.shape

In [ ]:
moddf.isna().any()[lambda x:x]

In [ ]:
del moddf[' Flow Packets/s']

In [ ]:
moddf.shape

In [ ]:
features=[" Destination Port" , "Bwd Packet Length Max", " Bwd Packet Length Mean" , " Bwd Packet Length Std", " Packet Length Mean", " URG Flag Count", " Average Packet Size", " Avg Bwd Segment Size"," Flow Duration"," Total Fwd Packets"," Subflow Bwd Packets"," Label"]

In [ ]:
len(features)

In [ ]:
X=moddf[features].copy()
X.head()

In [ ]:
giris = X.iloc[:,0:10]
cikis = X.iloc[:,-1]

In [ ]:
print(cikis)

In [ ]:
from sklearn.model_selection import train_test_split as tts
xtrain, xtest, ytrain, ytest = tts(giris, cikis, test_size=0.2)
xtrain

In [ ]:

import tensorflow as tf
from keras.layers import Dense, BatchNormalization, Dropout, LSTM, Bidirectional
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
#Early stopping
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [ ]:
x_train , y_train = np.array(xtrain), np.array(ytrain)
x_train = np.reshape(x_train, (xtrain.shape[0] , xtrain.shape[1], 1) )

In [ ]:
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=10, return_sequences=True, input_shape = (x_train.shape[1],1) ) )) #çıktı uzayının boyutu
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 10 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 10 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 10))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1,activation='sigmoid'))
regressor.compile(optimizer='adam', loss='binary_crossentropy',metrics=['acc'])
regressor.fit(x_train, y_train, epochs=10,batch_size=128 )

In [ ]:
trainloss=regressor.evaluate(x_train,y_train,verbose=0)

In [ ]:
x_test , y_test = np.array(xtest), np.array(ytest)
x_test = np.reshape(x_test, (xtest.shape[0] , xtest.shape[1], 1) )

In [ ]:
testloss=regressor.evaluate(x_test,y_test,verbose=0)

In [ ]:
predict = regressor.predict(x_test)

In [ ]:
xtest.shape[1]

In [ ]:
predict

In [ ]:
predDf = pd.DataFrame(y_test,columns=["Real predictions:"])

In [ ]:
predDf

In [ ]:
predDf2 = pd.Series(predict.reshape(45149,))
predDf2

In [ ]:
pred_results=pd.concat([predDf,predDf2],axis=1)
pred_results

In [ ]:
pred_results.columns=['Real_Values', 'Predicted_Values']

In [ ]:
pred_results

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
errorAb=mean_absolute_error(pred_results["Real_Values"], pred_results["Predicted_Values"])
errorSQ=mean_squared_error(pred_results["Real_Values"], pred_results["Predicted_Values"])

In [ ]:
errorAb

In [ ]:
errorSQ

In [ ]:
predict

In [ ]:
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=10, return_sequences=True, input_shape = (x_train.shape[1],1) ) )) #çıktı uzayının boyutu
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 10 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 10 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 10))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1,activation='sigmoid'))
regressor.compile(optimizer='adam', loss='binary_crossentropy',metrics=['acc'])
history=regressor.fit(x_train, y_train, epochs=3,batch_size=64 )

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()